In [1]:
import pandas as pd
import re

import os

import matplotlib.pyplot as plt
%matplotlib inline

**Oktober 2018**

Den här filen använder en del av koden som byggdes inför valet 2018. Målsättningen här är att kontrollera ifall Miljöpartiet "straffas" hårdare i de kommuner de styrt tillsammans med allianspartier, jämfört med de kommuner där de styrt tillsmamans med vänsterstyren. Bakgrunden är att det för tillfället förs en diskussion om MP:s agerande i Stockholm där partiet lämnade det rödgröna samarbetet och ingick i ett samarbete med Alliansen. Det förs därför en diskussion om att MP kommer behöva hantera en eventuell svekdebatt, då de avviker från väljarnas uppfattning att partiet de facto är ett vänsterparti. 

**Resultat**

Enligt vår sammanräkning så framgår det inget mönster i att MP:s väljare straffar partiet hårdare ifall det ingått i styren på högerkanten jämfört vänsterstyren. Sammanställning bygger på siffror från mandatperioderna 2006-2018. Artikeln publicerades i tidning nr 44. [Här är en länk till texten på Dagens Samhälles hemsida](https://www.dagenssamhalle.se/nyhet/mp-valjare-straffar-inte-blockbyten-25070).

### Hämta och formattera grunddata

----

#### Hämta grunddata

Först skapa rätt mappar:

In [2]:
from startup_tools import folder_maker

In [3]:
folder_maker(years=['2010','2014','2018'])

Nu hämta alla grundfiler från valmyndighetens API (kan ta ett par minuter)):

In [4]:
from startup_tools import xml_data_fetcher

In [5]:
%%time
xml_data_fetcher(count_type='slutresultat')

CPU times: user 5.79 s, sys: 3.83 s, total: 9.63 s
Wall time: 2min 13s


Nu extrahera formetterad valdata:

In [6]:
from startup_tools import ExtractData
E = ExtractData()

In [7]:
years = ['2010','2014','2018']

for year in years:
    E.fast_elec_calc(year=year,count_type="slutresultat")
    E.fast_particip_calc(year=year,count_type="slutresultat")
    #E.muni_elec_meta_data(year=year)

E.all_parties(count_type="slutresultat")
E.macro_results(count_type="slutresultat") 

**Klart!** All hämtad och formatterad data ligger nu i mappen data, och dess undermappar.

---

#### Formattering av grunddata

Eftersom vi önskar ta reda på skillnaderna mellan vad man fick för mandat in i en mandatperiod i respektive kommun, kontra vad man sedan får i samma kommuner valet därpå, då måste dessa få korrekt namn så vi kan skilja dem emellan. I följande beräkningar så syftar "\_väljarmandat" på det valresultat som MP fick i de kommuner där de var med och styrde _i den rådande mandatperioden som man utgår ifrån_. "\_valresultat" syftar sedermera på vad man sedan fick i valet därpå, dvs ifall man i dessa styren straffades eller belönades i de kommunerna där man ingick i ett styre.

In [3]:
def col_shaper(cols):
    """Den här funktionen tar våra dataframes kolumner och omformar \
dessa till en dictionary med önskad utformning. Så alla resultat \
från denna funktion vet att varannan kolumn bygger på äldre \
valresultat. Denna funktion ger varannan kolumn ändelsen \
'_väljarmandat' och varannan '_valresultat' i en dictionary. \
Se cellen direkt nedanför för exempel."""
    
    # det är endast de kolumner med valresultatsdata som ska formatteras, 
    # dessa har alltid en årsändelse, t ex "mandat_2006"
    formatted_cols = [col for col in cols if '_20' in col]
    
    # De övriga ska inte formatteras
    old_cols = [col for col in cols if '_20' not in col]
    
    # sortera kolumnerna som ska formatteras
    formatted_cols.sort()
    
    # spara hur de oformatterade kolumnerna ser ut
    olds = formatted_cols.copy()
    
    # lägg till de kolumner som inte ska formatteras
    [olds.append(col) for col in old_cols]
    
    counter = 1
    for i in range(len(formatted_cols)):
        # Hitta alla kolumner som har en sifferändelse i sig i 
        # listan på kolumner som ska formatteras
        r = re.findall("\w+(_\d+)",formatted_cols[i])[0]
        
        if counter % 2 != 0:
            # Ifall det är 1a, 2a, 3e etc kolumnen, gör såhär:
            formatted_cols[i] = formatted_cols[i].replace(r,"_väljarmandat")
        else:
            # annars gör såhär:
            formatted_cols[i] = formatted_cols[i].replace(r,"_valresultat")
        counter += 1
    
    # Lägg till de kolumner som inte skulle formatteras till listan på de
    # formatterade kolumnerna:
    [formatted_cols.append(col) for col in old_cols]
    
    # återge en dictionary på resultat:
    return dict(zip(olds,formatted_cols))
        

### Grunddata för alla år

In [4]:
from util import gov_mandates,block_gov_count

Funktionen gov_mandates() (kort för "government mandates") hämtar info om vilka som styrde respektive kommun mandatperioden från det angivna året. Alltså: vilka styrde? Och på vilket mandat styrde dem? Funktionen block_gov_count() lägger sedan till information om hur det gick efterföljande val.

In [5]:
# Ett exempel på vad funktionen hämtar:
gov_mandates(2014).head()

,kommun,parti,röster,mandat,procent_2014,block,summa_mandat_2014,summa_röster_2014
0,Ale,S,6032.0,17.0,33.59,V,49,18217
1,Ale,V,1122.0,3.0,6.25,V,49,18217
2,Ale,MP,1023.0,3.0,5.70,V,49,18217
3,Alingsås,M,5619.0,10.0,21.58,A,51,26369
4,Alingsås,L,2682.0,5.0,10.30,A,51,26369


In [6]:
%%time
frames = []
for year in [2010,2014,2018]:
    compare_year = year-4
    govs = gov_mandates(compare_year)
    tmp = block_gov_count(govs,value='mandat',elec_year=year,compare_year=compare_year,research_data=True)
    tmp['mandatperiod'] = f'{str(compare_year)}_{str(year)}'
    frames.append(tmp)

CPU times: user 13.9 s, sys: 47.9 ms, total: 14 s
Wall time: 14 s


### Beräkningar

In [7]:
from util import all_elec_years

Lägg ihop alla dataframes med valdata i en lista vi kan loopa igenom:

In [8]:
alla_perioder = pd.DataFrame()
for frame in frames:
    # Döp om alla kolumner:
    frame.columns = frame.columns.map(col_shaper(frame.columns))
    
    # Sortera fram alla alliansstyren där MP ingick och lägg till i "alla_perioder"
    alla_perioder=pd.concat([alla_perioder,
                             frame.loc[(frame['block']=='A')&(frame['parti']=='MP')]])
    
    # Sortera fram alla vänsterstyren där MP ingick och lägg till i "alla_perioder"
    alla_perioder=pd.concat([alla_perioder,
                             frame.loc[(frame['block']=='V')&(frame['parti']=='MP')]])


In [9]:
valresultat = alla_perioder.mandatperiod.unique().tolist()

Här endast för att döpa om mandatperioderna efter respektive valår:

In [10]:
alla_perioder.mandatperiod=alla_perioder.mandatperiod.map(dict(zip(valresultat,[2006,2010,2014,2018])))

Eftersom vi enbart sorterat fram alla rader i totaldatat som är MP:s data så kan vi skippa partikolumnen helt och hållet. 

Samuel bad om procentutveckling i alla styren där MP ingår. Därför nöjer vi oss med de kolumner som krävs för att få fram detta:

In [11]:
resultat = alla_perioder.loc[:,['kommun',
                                'mandatperiod',
                                'block',
                                'röster_väljarmandat',
                                'summa_röster_väljarmandat',
                                'röster_valresultat',
                                'summa_röster_valresultat']]

In [12]:
resultat.iloc[:,3:] = resultat.iloc[:,3:].astype('int')

In [13]:
resultat=resultat.groupby(['mandatperiod','block']).sum().reset_index()

In [14]:
resultat

,mandatperiod,block,röster_väljarmandat,summa_röster_väljarmandat,röster_valresultat,summa_röster_valresultat
0,2006,A,24523,595128,33988,635483
1,2006,V,60289,1068082,84981,1145118
2,2010,A,41879,773367,48330,807239
3,2010,V,112622,1626120,126743,1699773
4,2014,A,38417,569635,23502,601812
5,2014,V,226600,2265896,143601,2384535


In [15]:
resultat.columns

Index(['mandatperiod', 'block', 'röster_väljarmandat',
       'summa_röster_väljarmandat', 'röster_valresultat',
       'summa_röster_valresultat'],
      dtype='object')

In [16]:
resultat['väljarmandat'] = ((resultat['röster_väljarmandat'] / resultat['summa_röster_väljarmandat'])*100).round(2)

In [17]:
resultat['valresultat'] = ((resultat['röster_valresultat'] / resultat['summa_röster_valresultat'])*100).round(2)

Kolumnen "resultat" är bara en rak procentjämförelse på hur många andelar högre eller lägre följande valresultat var inom respektive blockstyre:

In [18]:
resultat['resultat'] = (((resultat['valresultat'] / resultat['väljarmandat'])-1)*100).round(2)

In [19]:
resultat=resultat.loc[:,['mandatperiod','block','väljarmandat','valresultat','resultat']]

In [20]:
resultat

,mandatperiod,block,väljarmandat,valresultat,resultat
0,2006,A,4.12,5.35,29.85
1,2006,V,5.64,7.42,31.56
2,2010,A,5.42,5.99,10.52
3,2010,V,6.93,7.46,7.65
4,2014,A,6.74,3.91,-41.99
5,2014,V,10.00,6.02,-39.80


In [22]:
resultat['skillnad'] = resultat['valresultat'] - resultat['väljarmandat']

In [23]:
resultat

,mandatperiod,block,väljarmandat,valresultat,resultat,skillnad
0,2006,A,4.12,5.35,29.85,1.23
1,2006,V,5.64,7.42,31.56,1.78
2,2010,A,5.42,5.99,10.52,0.57
3,2010,V,6.93,7.46,7.65,0.53
4,2014,A,6.74,3.91,-41.99,-2.83
5,2014,V,10.00,6.02,-39.80,-3.98


In [24]:
os.makedirs('res')

In [25]:
resultat.to_excel('res/sammanfattning.xlsx',index=False)

In [26]:
alla_perioder=alla_perioder[['kommun','mandatperiod','block','procent_väljarmandat','procent_valresultat']].reset_index(drop=True)

In [27]:
alla_perioder['resultat_i_följande_val'] = alla_perioder.procent_valresultat-alla_perioder.procent_väljarmandat

In [28]:
alla_perioder.to_excel('res/alla_kommuner.xlsx',index=False)